# Big Data w biznesie

## Czwarty notebook rozwiązania

In [ ]:
from IPython.core.display import HTML


def _set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter.
    Pass the file path to the CSS file.
    """

    styles = open(css_file_path, "r").read()
    s = '<style>%s</style>' % styles
    return HTML(s)


_set_css_style("../custom.css")

#### Zaimportuj biblioteki numpy, pandas, matplotlib i seaborn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

#### Załaduj plik csv.
***
Do zmiennej `df` załaduj plik `pokemon.csv`.

In [ ]:
df = pd.read_csv('./pokemon.csv')

Ten zbiór danych zawiera informacje na temat 801 Pokémonów z pierwszych siedmiu generacji. Obejmuje on podstawowe statystyki, osiągi przeciwko innym typom, wzrost, wagę, klasyfikacje punkty doświadczenia, zdolności itp. Informacje zostały pobrane z serwisu serebii (www.serebii.net) i mają następujące cechy:

- abilities: ciąg znaków zawierający listę zdolności, które Pokémon może posiadać.
- against_?: osiemnaście cech, które określają ilość otrzymywanych obrażeń od ataku określonego typu Pokémona.
- attack: podstawowy atak Pokémona.
- base_egg_steps: liczba kroków wymagana do wyklucia się jaja Pokémona.
- base_happiness: podstawowe szczęście Pokémona.
- base_total: suma hp, ataku, obrony, specjalnego ataku, specjalnej obrony i prędkości.
- capture_rate: współczynnik łapania Pokémona.
- classification: klasyfikacja Pokémona, opisana przez Pokédex Sun and Moon.
- defense: podstawowa obrona Pokémona.
- experience_growth: wzrost doświadczenia Pokémona.
- height_m: wysokość Pokémona w metrach.
- hp: podstawowe HP Pokémon. Jest to skrót od Hit Point, który określa, ile obrażeń Pokémon może otrzymać przed omdleniem.
- japanese_name: oryginalna japońska nazwa Pokémona.
- name: angielska nazwa Pokémona.
- percentage_male: procent osobników płci męskiej w gatunku. Puste, jeśli Pokémon jest bezpłciowy.
- pokedex_number: numer wpisu Pokémona w Krajowym Pokédexie.
- sp_attack: podstawowy specjalny atak Pokémona.
- sp_defense: podstawowa specjalna obrona Pokémona.
- speed: podstawowa prędkość Pokémona.
- type1: podstawowy typ Pokémona.
- type2: drugi typ Pokémona.
- weight_kg: waga Pokémona w kilogramach.
- generation: numerowana generacja, w której wprowadzono Pokémona.
- is_legendary: oznacza, czy Pokémon jest legendarny.

In [ ]:
df.describe().T

#### Przygotuj dane.
***
Wykonamy tu następujące czynności:
- wyrzucimy nieinteresujące na kolumny
- zmienimy nazwy kolumny na zaczynające się wielką literą
- wstawimy kolumnę z nazwą Pokémonów na początek
- zmienimy wartości w kolumnie `'Is_Legendary'` na napisy
- zmienimy nazwy typów Pokémonów na zaczynające się wielką literą
- zmienimy dane w kolumnie `'Capture_Rate'` na liczbowe

In [ ]:
df = df.drop(
    ['japanese_name', 'pokedex_number', 'base_egg_steps', 'classfication', 'percentage_male'],
    axis=1
)

df = df.rename(str.title, axis='columns')

df = df.reindex(['Name'] + [col for col in df.columns if col != 'Name'], axis=1)

df = df.replace(
    {
        'Is_Legendary': {
            0: "Non-legendary",
            1: "Legendary",
        },
    },
)

types = [t for t in list(df['Type1'].unique())]
df['Type1'] = df['Type1'].replace(types, [t.title() for t in types])
df['Type2'] = df['Type2'].replace(types, [t.title() for t in types])


df['Capture_Rate'] = df['Capture_Rate'].replace({'30 (Meteorite)255 (Core)': np.nan})
df['Capture_Rate'] = pd.to_numeric(df['Capture_Rate'])

#### Dodaj parametry do Seaborna.
***
Dodanie parametrów rysowania w Seabornie pozwala na dostosowanie wyglądu wykresów do naszych preferencji i wymagań.
Ustawienie tych wartości przed utworzeniem wykresów pozwala na utrzymanie spójnego wyglądu między różnymi wykresami.

In [ ]:
SEABORN_RC_PARAMS = {
    'font.size': 6,
    'xtick.color': 'gray',
    'xtick.labelsize': 6,
    'ytick.color': 'gray',
    'ytick.labelsize': 6,
    'text.color': 'gray',
    'legend.fontsize': 6,
    'legend.loc': 'upper left',
    'legend.frameon': False,
    'xaxis.labellocation': 'left',
    'yaxis.labellocation': 'top',
    'axes.edgecolor': 'gray',
    'axes.labelcolor': 'gray',
    'axes.titlelocation': 'left',
    'axes.labelsize': 6,
    'axes.titlesize': 12,
    'axes.titlepad': 20,
    'figure.figsize': (6, 3),
    'figure.dpi': 200,
    'figure.titlesize': 12,
}
SEABORN_STYLE = 'white'
sns.set(rc=SEABORN_RC_PARAMS, style=SEABORN_STYLE)

### Rysowanie ładnych wykresów.
***
Poniżej przedstawiam kilka inspirujących przykładów wykresów, które można stworzyć na podstawie danych dotyczących Pokémonów. Przy każdym z tych wykresów warto zwrócić uwagę na kilka elementów:

- W jaki sposób dane są modyfikowane przed stworzeniem wykresu?
- Jaki rodzaj wykresu jest używany i dlaczego taki wybór został dokonany?
- Jakie metody i funkcje są używane do stworzenia danego wykresu?
- Jakie argumenty przyjmują te metody i funkcje?

Jeśli pojawiają się jakieś wątpliwości co do działania danej metody lub argumentu, warto spróbować usunąć go z kodu i zobaczyć, jakie zmiany wpłyną na wykres.

Wykresy te powstały na podstawie notebooka dostępnego pod adresem https://www.kaggle.com/code/joaopdrg/discovering-the-best-pok-mon, gdzie znajduje się więcej przykładów.

#### Przeanalizuj rysowanie wykresu słupkowego z liczbą wystąpień.
***
Wykres przedstawia liczbę nowych Pokémonów w każdej generacji.

In [ ]:
ax = sns.countplot(y="Generation", data=df, orient='h', palette=['indianred', 'indianred', 'indianred', 'indianred', 'darkred', 'indianred', 'indianred'])
ax.set_title('New Pokémon by Generation')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks(ax.get_xticks(), labels='')
sns.despine(top=True, bottom=True, left=True, right=True)
ax.bar_label(ax.containers[0], fmt='%0.f')
plt.show()

#### Przeanalizuj rysowanie wykresu słupkowego.
***
Wykres przedstawia łączną liczbę Pokémonów każdego typu.

In [ ]:
type1_value_counts = df['Type1'].value_counts()
type2_value_counts = df['Type2'].value_counts()
types_df = pd.concat([type1_value_counts, type2_value_counts], axis=1)
types_df.sort_values(by='Type2', ascending=True, inplace=True)

In [ ]:
ax = types_df.plot(
        kind= 'barh',
        stacked= True,
        color= ['darkred', 'indianred'],
        width= 1
)
ax.set_title('Pokémon by Primary and Secondary Types')
ax.set_xlabel('Count', loc='right')
sns.despine(top=True, left=True, right=True)
for container in ax.containers:
    ax.bar_label(
            container,
            fmt= '%0.f',
            color='white',
            label_type= 'center'
    )

#### Przeanalizuj rysowanie pudełkowego.
***
Wykres powinien przedstawiać zależność współczynnika łapania Pokémonów dla każdego typu z podziałem na Pokémony legendarne.

In [ ]:
ax=sns.boxplot(
        df,
        x= 'Type1',
        y= 'Capture_Rate',
        hue= 'Is_Legendary',
        palette= ['darkred', 'indianred']
    )
ax.set_title('Pokémon Capture Rate by Type')
plt.ylim(bottom=0)
plt.xticks(rotation=90)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
sns.despine()

#### Przeanalizuj rysowanie wykresu punktowego.
***
Wykres przedstawia zależność wzrostu od wagi dla każdego Pokémona z podziałem na legendarne.

In [ ]:
sns.scatterplot(
    df,
    x= 'Weight_Kg',
    y= 'Height_M',
    hue= 'Is_Legendary',
    palette= ['darkred', 'indianred']
)
ax.set_title('Pokémon by Weight and Height')
sns.despine()
plt.legend(frameon=True, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.5)
legend_kws={
    'borderpad': 0,
    'markerscale': 0.5,
    'handlelength': 0,
},
plot_average=False

# Label top 5 by weight and height
top5_weight_height_merged = pd.concat([
    df.nlargest(5, 'Height_M'),
    df.nlargest(5, 'Weight_Kg'),
]).drop_duplicates(subset=['Name'])
for index, row in top5_weight_height_merged.iterrows():
    plt.annotate(row['Name'], xy=(row['Weight_Kg']+10, row['Height_M']))

#### Przeanalizuj rysowanie mapy ciepła.
***
Wykres przedstawia efektywność każdego typu Pokémonów.

In [ ]:
against_columns = [
    'Against_Bug',
    'Against_Dark',
    'Against_Dragon',
    'Against_Electric',
    'Against_Fairy',
    'Against_Fight',
    'Against_Fire',
    'Against_Flying',
    'Against_Ghost',
    'Against_Grass',
    'Against_Ground',
    'Against_Ice',
    'Against_Normal',
    'Against_Poison',
    'Against_Psychic',
    'Against_Rock',
    'Against_Steel',
    'Against_Water',
]

pokemon_type_effectiveness = df.groupby(['Type1']).median(numeric_only=True)[against_columns]

In [ ]:
ax = sns.heatmap(
    pokemon_type_effectiveness,
    annot= True,
    cbar= False,
    cmap= 'Reds',
    fmt= '.1f',
    vmin= 0,
    vmax= 4
)
ax.set_title('Effectiveness Type Chart')
ax.set_xlabel('')
ax.set_ylabel('')
ax.xaxis.tick_top()
plt.tick_params(top=False)
plt.xticks(rotation=90)
plt.show()

#### Narysuj wykres pudełkowy.
***
Wykres przedstawia zależność współczynnika łapania Pokémonów dla każdej generacji z podziałem na Pokémony legendarne.

In [ ]:
ax=sns.boxplot(
        df,
        x= 'Generation',
        y= 'Capture_Rate',
        hue= 'Is_Legendary',
        palette= ['darkred', 'indianred']
    )
ax.set_title('Pokémon Capture Rate by Generation')
plt.ylim(bottom=0)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
sns.despine()

#### Narysuj wykres słupkowy.
***
Stwórz wykres słupkowy, który przedstawia top 10 Pokémonów z najlepszym łącznym wynikiem bazowych statystyk (`'Base_Total'`).

In [ ]:
top10_pokemon_base_total = df.loc[:, ['Name', 'Base_Total']].sort_values(
    by='Base_Total',
    ascending=False,
)[:10].reset_index()

In [ ]:
ax = sns.barplot(x="Base_Total", y="Name", data=top10_pokemon_base_total, orient='h', color= 'darkred')
ax.set_title('Pokémon With Best Base Total')
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks(ax.get_xticks(), labels='')
sns.despine(top=True, bottom=True, left=True, right=True)
ax.bar_label(ax.containers[0], fmt='%0.f')
plt.show()

#### Usuń niepotrzebne kolumny.
***
Zanim przejdziemy do dalszego etapu, gdzie będziemy wizualizować dane za pomocą widgetów, wyrzućmy 20 początkowych kolumn (poza pierwszą kolumną `'Name'`), które nie zawierają ważnych informacji.

Dla zainteresowanych te kolumny to:

`['Abilities', 'Against_Bug', 'Against_Dark', 'Against_Dragon', 'Against_Electric', 'Against_Fairy', 'Against_Fight', 'Against_Fire', 'Against_Flying', 'Against_Ghost', 'Against_Grass', 'Against_Ground', 'Against_Ice', 'Against_Normal', 'Against_Poison', 'Against_Psychic', 'Against_Rock', 'Against_Steel', 'Against_Water']`

In [ ]:
df = df.drop(df.iloc[:,1:20], axis=1)

#### Stwórz Dashboard do wyświetlania i filtrowania DataFrame'a.
***
Chcemy stworzyć dashboard bardzo podobny do tego, który stworzyliśmy w notebooku `Big_Data_4.ipynb`, ale tym razem na danych dotyczących Pokémonów.

Najpierw potrzebujemy dwóch rozwijanych list: `'Generation'` (generacja) oraz `'Type1'` (pierwotny typ).
Uważaj na to, czy nazwy kolumn rozpoczynają się z wielkiej, czy małej litery.

Potrzebujemy również pola tekstowego do wprowadzania liczb z ograniczeniami, które będzie zmieniać kolorowanie kolumn: `['Attack', 'Defense', 'Sp_Attack', 'Sp_Defense', 'Speed']`.

Dodatkowo potrzebujemy wyjścia, które będzie wyświetlać przefiltrowany DataFrame (`output`), oraz wyjścia, które będzie wyświetlać wykres KDE (Kernel Density Estimation) dla kolumny `'Base_Total'` (`plot_output`).

Chcemy również stworzyć funkcję do filtrowania danych, która pobiera wartości z trzech widgetów i filtruje dane z DataFrame'a na podstawie wybranych wartości.

Następnie dodamy trzy funkcje reagujące na zdarzenia i trzy funkcje obserwujące widgety, które wywołują funkcję `common_filtering` z odpowiednimi argumentami, gdy wartość widgetu zostanie zmieniona.

Na koniec połączymy wszystkie widgety i wyjścia w jeden dashboard. W jednej zakładce wyświetlany będzie DataFrame, a w drugiej wykres KDE.


In [ ]:
def unique_sorted_values_plus_all(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, 'All')
    return unique

def colour_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

item_layout = widgets.Layout(margin='0 0 50px 0')

dropdown_generation = widgets.Dropdown(options = unique_sorted_values_plus_all(df['Generation']))
dropdown_type1 = widgets.Dropdown(options = unique_sorted_values_plus_all(df['Type1']))
bounded_num = widgets.BoundedFloatText(min=0, max=200, value=100, step=5)
input_widgets = widgets.HBox([dropdown_generation, dropdown_type1, bounded_num], layout=item_layout)



output = widgets.Output()
plot_output = widgets.Output()

def common_filtering(generation, type1, num):
    output.clear_output()
    plot_output.clear_output()

    if generation == 'All': generation = df['Generation'].unique().tolist()
    if type1 == 'All': type1 = df['Type1'].unique().tolist()

    common_filter = df.query('Generation == @generation and Type1 == @type1')

    with output:
        display(common_filter.style.applymap(
                    lambda x: colour_ge_value(x, num),
                    subset=['Attack', 'Defense', 'Sp_Attack', 'Sp_Defense', 'Speed']))

    with plot_output:
        sns.kdeplot(common_filter['Base_Total'], fill=True)
        plt.show()

def dropdown_generation_eventhandler(change):
    common_filtering(change.new, dropdown_type1.value, bounded_num.value)

def dropdown_type1_eventhandler(change):
    common_filtering(dropdown_generation.value, change.new, bounded_num.value)

def bounded_num_eventhandler(change):
    common_filtering(dropdown_generation.value, dropdown_type1.value, change.new)

dropdown_generation.observe(
dropdown_generation_eventhandler, names='value')
dropdown_type1.observe(
dropdown_type1_eventhandler, names='value')
bounded_num.observe(
bounded_num_eventhandler, names='value')

tab = widgets.Tab([output, plot_output], layout=item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')

dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

#### Stwórz Dashboard do rysowania i zmieniania wykresów.
***
Naszym drugim zadaniem jest opracowanie Dashboardu, który umożliwi nam tworzenie wykresów na podstawie dostępnych danych oraz zapewni możliwość modyfikowania właściwości wykresów. W naszym Dashboardzie chcemy uwzględnić następujące elementy (widgetów):

- Wywoływanie funkcji rysującej
    - Przycisk - służy do wywołania funkcji rysującej wykres.

- Wybór danych na obu osiach (tylko dane numeryczne!)
    - Rozwijana lista - służy do wyboru kolumny zawierającej dane dla osi X na wykresie.

    - Rozwijana lista - służy do wyboru kolumny zawierającej dane dla osi Y na wykresie.

- Nadawanie tytułu i nazw osi
    - Pole tekstowe - służy do wprowadzenia tytułu wykresu.

    - Pole tekstowe - służy do wprowadzenia nazwy osi X.

    - Pole tekstowe - służy do wprowadzenia nazwy osi Y.

- Usuwanie obramowania (`despine`)
    - Zestaw czterech pól wyboru - służą do wyboru, które obramowania wykresu mają być usunięte (opcje to "Top", "Bottom", "Left" i "Right").


-  Ustalanie zakresu danych na osiach
    - Suwak - służy do ustawienia zakresu osi X na wykresie w procentach.

    - Suwak - służy do ustawienia zakresu osi Y na wykresie w procentach.

    - Pole wyboru - służy do wyboru, czy ustawić zakres osi X od 0.

    - Pole wyboru - służy do wyboru, czy ustawić zakres osi Y od 0.

-  Stylowanie
    - Pole wyboru - służy do obrotu etykiet osi x.

    - `ColorPicker` - służy do wyboru koloru wykresu.

    - Przycisk - służy do włączania i wyłączania siatki na wykresie.

    - Przyciski przełączające (`ToggleButtons`) - służą do zmiany stylu wykresu. Opcje to 'white', 'dark', 'whitegrid', 'darkgrid', 'ticks'.

In [ ]:
button = widgets.Button(
    description='Plot',
)

x_dropdown = widgets.Dropdown(options = df.select_dtypes(include=np.number).columns.tolist())

x_title = widgets.HTML('<h5>Choose Data for X:</h5>')

y_dropdown = widgets.Dropdown(options = df.select_dtypes(include=np.number).columns.tolist())

y_title = widgets.HTML('<h5>Choose Data for Y:</h5>')

title_textbox = widgets.Text(
    placeholder='Wpisz tytuł wykresu...',
    description='Title:',
    indent=False
)

x_textbox = widgets.Text(
    placeholder='Wpisz tytuł osi x...',
    description='X Axis Title:',
    indent=False
)

y_textbox = widgets.Text(
    placeholder='Wpisz tytuł osi y...',
    description='Y Axis Title:',
    indent=False
)

despine_title = [widgets.HTML('<h5>Despine:</h5>')]

despine_types = ["Top", "Bottom", "Left", "Right"]

despine_layout = widgets.Layout(margin='0px 0px')

despine_checkboxes = [widgets.Checkbox(value=False, description=label, layout=despine_layout, indent=False) for label in despine_types]

despine_VBox = widgets.VBox(children=despine_title+despine_checkboxes)

xlim_slider = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=1,
    description='Xlim %:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

ylim_slider = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=1,
    description='Ylim %:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

xlim_checkbox = widgets.Checkbox(
    value=False,
    description="Xlim from 0",
    indent=False
)

ylim_checkbox = widgets.Checkbox(
    value=False,
    description="Ylim from 0",
    indent=False
)

rotation_checkbox = widgets.Checkbox(
    value=False,
    description="Rotate X Ticks",
    indent=False
)

plot_color_picker = widgets.ColorPicker(
    concise=True,
    description='Plot color:',
    value='#91C9FE',
)

grid_button = widgets.ToggleButton(
    value=False,
    description='Grid',
    icon='check'
)

style_buttons = widgets.ToggleButtons(
    options=['white', 'dark', 'whitegrid', 'darkgrid', 'ticks'],
    description='Theme:',
)

output_pokemon = widgets.Output()

Teraz musimy napisać funkcję, która będzie wywoływana przy naciśnięciu przycisku i będzie rysować nasz wykres.

W dużym uproszczeniu funkcja powinna sprawdzać wartości poszczególnych widgetów i w zależności od ich wartości modyfikować sposób rysowania funkcji.

Co do rodzaju rysowanego wykresu, pozostawiam dowolność, aczkolwiek na początek polecam `regplot` albo `scatterplot`, ponieważ nie powinno być z nimi problemu, jeśli chodzi o rodzaj danych. Potem oczywiście można bardzo łatwo zmienić rodzaj wykresu albo dodać możliwość wyboru rodzaju wykresu.

Podpowiedź
Przyda się napisanie funkcji pomocniczej, która pobiera wartości pól wyboru, które obramowania wykresu mają być usunięte (`despine`). Funkcja powinna zwracać słownik, który wygląda mniej więcej tak:

`słownik = {"top":True, "bottom":False, "left":False, "right":True}`

Dzięki takiemu słownikowi można łatwo wywołać funkcję, która usuwa obramowanie:

`sns.despine(**słownik)`

In [ ]:
def check_despine(despine_checkboxes):
    despine_dict = {"top":False, "bottom":False, "left":False, "right":False}
    for checkbox in despine_checkboxes:
        if checkbox.value:
            despine_dict[checkbox.description.lower()] = True
    return despine_dict

In [ ]:
def plot(change=None):
    output_pokemon.clear_output()
    despine_dict = check_despine(despine_checkboxes)
    x_data = x_dropdown.value
    y_data = y_dropdown.value
    title = title_textbox.value
    xlabel = x_textbox.value
    ylabel = y_textbox.value
    rotation = rotation_checkbox.value
    plot_color = plot_color_picker.value
    style = style_buttons.value
    grid = grid_button.value
    set_0_xlim = xlim_checkbox.value
    set_0_ylim = ylim_checkbox.value
    x_range = (df[x_data].max() - df[x_data].min())
    y_range = (df[y_data].max() - df[y_data].min())
    xlim = (df[x_data].min()+x_range*(xlim_slider.value[0]/100), df[x_data].min()+x_range*(xlim_slider.value[1]/100))
    ylim = (df[y_data].min()+y_range*(ylim_slider.value[0]/100), df[y_data].min()+y_range*(ylim_slider.value[1]/100))
    with output_pokemon:
        sns.set(style=style)
        ax = sns.regplot(x=x_data, y=y_data, data=df, color=plot_color)
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        if rotation:
            plt.xticks(rotation=90)
        if set_0_xlim:
            ax.set_xlim(0,)
        else:
            ax.set_xlim(xlim)
        if set_0_ylim:
            ax.set_ylim(0,)
        else:
            ax.set_ylim(ylim)
        ax.grid(grid)
        sns.despine(**despine_dict)
        plt.show()

In [ ]:
button.on_click(plot)

Na końcu naszym celem jest połączenie trzech elementów w jeden spójny Dashboard.

Pierwszy element to zestaw widgetów, które pozwolą nam na modyfikowanie właściwości wykresów. Planujemy podzielić je na dwie grupy i umieścić w oddzielnych zakładkach - Plot oraz Styling.

Drugi element to przycisk, który po kliknięciu wywoła generowanie wykresu.

Trzeci element to wyjście (Output), gdzie będzie wyświetlany nasz wygenerowany wykres.

In [ ]:
tab1_pokemon = widgets.VBox(children=[title_textbox,
                              x_title,
                              x_dropdown,
                              widgets.HBox(children=[xlim_checkbox, xlim_slider]),
                              x_textbox,
                              rotation_checkbox,
                              y_title,
                              y_dropdown,
                              widgets.HBox(children=[ylim_checkbox, ylim_slider]),
                              y_textbox,
                              ])
tab2_pokemon = widgets.VBox(children=[style_buttons,
                              despine_VBox,
                              grid_button,
                              plot_color_picker,
                              ])
tab_pokemon = widgets.Tab(children=[tab1_pokemon, tab2_pokemon])
tab_pokemon.set_title(0, 'Plot')
tab_pokemon.set_title(1, 'Styling')
dashboard_pokemon = widgets.VBox(children=[tab_pokemon, button, output_pokemon])
display(dashboard_pokemon)